<a href="https://colab.research.google.com/github/sunshineluyao/waiting-time-eip1559/blob/main/code/Data_Query/Query_Data_MemopoolGuru_Public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference:
* https://fanzzz.notion.site/Instructions-to-access-mempool-guru-data-ab8977977d4b48e68a27cae3af545d2d
* https://www.psycopg.org/psycopg3/docs/basic/usage.html
* https://www.psycopg.org/psycopg3/docs/basic/transactions.html

!!!Note, please remove credentials before circulate 

# Part I: Set-up

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# print the current directory
!pwd

/content


In [ ]:
#@title path
#create a project folder and identify the path
path = "/content/drive/MyDrive/waitingtime" #@param {type:"string"}

     

In [ ]:
#go to your project folder 
%cd /content/drive/MyDrive/waitingtime

/content/drive/MyDrive/waitingtime


In [ ]:
# print the current directory again to make sure that you are in the project folder
!pwd     

/content/drive/MyDrive/waitingtime


## 1.1. Install and import packages

references:

https://www.psycopg.org/psycopg3/docs/

In [ ]:
#import Psycopg 3 – PostgreSQL database adapter for Python
!pip install psycopg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import Psycopg 3 – PostgreSQL database adapter for Python
import psycopg

## 1.2. connect to the server using private credentials

In [ ]:
#@title username
# security alert: privacy information, please do not share or distribute without permission
username = "username" #@param {type:"string"}

In [ ]:
#@title password
# security alert: privacy information, please do not share or distribute without permission
password = "passowrd" #@param {type:"string"}


In [ ]:
#@title IP Address
# security alert: privacy information, please do not share or distribute without permission
IP_Address = "IP" #@param {type:"string"}


In [ ]:
#Sample code: POSTGRES_DB_URL = f"postgresql://{username}:{password}@{address}/tx_log"
#Action 1: replace {username}, including the parenthesis, with your username provided in the credential document
#Action 2: replace {password}, including the parenthesis, with your password provided in the credential document
#Action 3: replace {address}, including the parenthesis, with your address provided in the credential document
# frequent mistakes: if you don't remove the patherentis, the code ends up with Python syntax error. Read more here: https://peps.python.org/pep-0498/
POSTGRES_DB_URL = f"postgresql://username:password@IP/tx_log"

# Part II: First Time Observation

In [ ]:
# define the function to identify the first serve and time that one transantion was found in the public mempool
def find_first_observation(cur, hash):
	return cur.execute("select monitor_id, time_seen, hash, block_hash from FirstObservation where hash = (%s)",
                        (hash, )).fetchall()

### Sample code for finding the set of servers that observes each transaction and the time first seen by each server using transaction hash

In [ ]:
#@title transaction hash of the transactions
# Go to etherscan or Google Bigquery Public database and input the transaction hash of one block that you are interested in: 
#https://etherscan.io/
#https://www.kaggle.com/datasets/bigquery/ethereum-blockchain
transaction_hash = "0x1cf33585fcb020xa4bcbfdd72f3ecc4edaf16253eb33c018934e110d959f5f9c56b8681eed6bea9" #@param {type:"string"}


In [ ]:
#print results for the block that you are interested in 
with psycopg.connect(POSTGRES_DB_URL, autocommit=True) as conn:
    with conn.cursor() as cur:
        for ob in find_first_observation(cur, "0xa4bcbfdd72f3ecc4edaf16253eb33c018934e110d959f5f9c56b8681eed6bea9"):
            print(f"first seen by {ob[0]} at {ob[1]} for {ob[2]} finally recorded in block_hash {ob[3]}")

# Part III: Waiting Time (Delay) Data Query

##3.1 Query the table named "processed_transactions"
### Data Dictionary
|name  |type| sample|
|---   |---|----|
|first_observed_at| timestamp with timezone|      |
|first observed_by| varchar(20)|               |
|included_in      |varchar(80)|                |
|included_at      |timestamp with timezone|    |
|delay| double precision|                    |
|num_observations|         integer|           |
|included_in_block_num| integer|             |
|hash|                |varchar(80)|           |

### A. Query an observation

In [ ]:
# Connect to an existing database
with psycopg.connect(POSTGRES_DB_URL, autocommit=True) as conn:

    # Open a cursor to perform database operations
    with conn.cursor() as cur:

        # Execute a command: this creates a new table
        print(cur.execute("select * from processed_transactions Limit 1").fetchone())

('0x7614ba7ea2f28a54540b8a535b4d55ae70349bfeb21bf5fc62e620cb9fa0fa83', datetime.datetime(2022, 9, 16, 4, 21, 2, 454719, tzinfo=backports.zoneinfo.ZoneInfo(key='Etc/UTC')), 'VA', '0x38e89e3bc65fc8662ac7dc4da0b5acf37a47ff1dbb7586883d8663a46811b6aa', datetime.datetime(2022, 9, 16, 4, 21, 25, 153676, tzinfo=backports.zoneinfo.ZoneInfo(key='Etc/UTC')), 22.698957, 3, 15543784)


### B. Print all the column names

In [ ]:
# Connect to an existing database
with psycopg.connect(POSTGRES_DB_URL, autocommit=True) as conn:

    # Open a cursor to perform database operations
    with conn.cursor() as cur:
      cur.execute("select * from processed_transactions LIMIT 0")
      column_names_processed_transactions =[desc[0] for desc in cur.description]
      print(column_names_processed_transactions)

['hash', 'first_observed_at', 'first_observed_by', 'included_in', 'included_at', 'delay', 'num_observations', 'included_in_block_num']


### C. Save to the datafrme

In [ ]:
import pandas as pd

In [ ]:
# Connect to an existing database
with psycopg.connect(POSTGRES_DB_URL, autocommit=True) as conn:

    # Open a cursor to perform database operations
    with conn.cursor() as cur:

        # Execute a command: this creates a new table
        sql=cur.execute("select * from processed_transactions limit 2").fetchmany(size=2)
        dat =pd.DataFrame(sql, columns=column_names_processed_transactions)

In [ ]:
dat

,hash,first_observed_at,first_observed_by,included_in,included_at,delay,num_observations,included_in_block_num
0,0x7614ba7ea2f28a54540b8a535b4d55ae70349bfeb21b...,2022-09-16 04:21:02.454719+00:00,VA,0x38e89e3bc65fc8662ac7dc4da0b5acf37a47ff1dbb75...,2022-09-16 04:21:25.153676+00:00,22.698957,3,15543784
1,0x778d22bafd3b7a4ab6015d103c6731578235388684de...,2022-09-16 04:21:12.028780+00:00,VA,0x38e89e3bc65fc8662ac7dc4da0b5acf37a47ff1dbb75...,2022-09-16 04:21:25.153676+00:00,13.124896,1,15543784


### define the column and blocks to be queried

In [ ]:
selected_columns = ['delay', 'included_in_block_num']

In [ ]:
# Connect to an existing database
with psycopg.connect(POSTGRES_DB_URL, autocommit=True) as conn:

    # Open a cursor to perform database operations
    with conn.cursor() as cur:

        # Execute a command: this creates a new table
        sql=cur.execute("select delay, included_in_block_num from processed_transactions WHERE included_in_block_num >= 15537293 and included_in_block_num <= 15537493 limit 2").fetchmany(size=2)
        dat =pd.DataFrame(sql, columns=selected_columns)

In [ ]:
dat

,delay,included_in_block_num
0,41.872186,15537305
1,2.960342,15537305


## 3.2. Before and After the Merge

Data Range: We are going to query waiting time (delay) data for those included in the ethereum blockchain before and after the merge (block number: 15537393 on September 15)

Reference; https://www.investopedia.com/ethereum-completes-the-merge-6666337

In [ ]:
#@title the merge
merge = 15537393 #@param {type:"number"}

In [ ]:
#@title data radius
radius = 70000 #@param {type:"number"}

In [ ]:
### find the start and the end
start = merge-radius
end = merge+radius
print(start, end)

15467393 15607393


In [ ]:
# Connect to an existing database
with psycopg.connect(POSTGRES_DB_URL, autocommit=True) as conn:

    # Open a cursor to perform database operations
    with conn.cursor() as cur:

        # Execute a command: this creates a new table
        sql=cur.execute("select delay, included_in_block_num from processed_transactions WHERE included_in_block_num >= 15467393 and included_in_block_num <= 15607393").fetchall()
        dat =pd.DataFrame(sql, columns=selected_columns)

In [ ]:
wt_merge=dat
wt_merge

,delay,included_in_block_num
0,6.961628,15543911
1,6.239315,15543911
2,13.702400,15543911
3,11.898386,15543911
4,12.329767,15543911
...,...,...
23006690,9.533371,15543911
23006691,9.454926,15543911
23006692,9.086987,15543911
23006693,8.778224,15543911


In [ ]:
wt_merge.to_csv('wt_merge.csv')

In [ ]:
# save the data file to pickle to reduce size
wt_merge.to_pickle('wt_merge.pkl')

In [ ]:
wt_merge =pd.read_pickle('wt_merge.pkl')
wt_merge  

,delay,included_in_block_num
0,6.961628,15543911
1,6.239315,15543911
2,13.702400,15543911
3,11.898386,15543911
4,12.329767,15543911
...,...,...
23006690,9.533371,15543911
23006691,9.454926,15543911
23006692,9.086987,15543911
23006693,8.778224,15543911


instructions:

https://www.analyticsvidhya.com/blog/2021/06/how-to-load-kaggle-datasets-directly-into-google-colab/

## 3.3. Three NFT drops during 2021-08-24 to 2021-09-21


* 2.1. Fatales NFT Launches (2021-08-31)
* 2.2. CryptoDads NFT Launches (2021-09-10)
* 2.3. GalaxyEggs NFT Launches (2021-9-14)




### A. From datetime to unix timestamp:
how-to-convert-datetime-to-unix-timestamp-in-python/


In [ ]:
import datetime
import time

In [ ]:
current_timestamp = time.time()

In [ ]:
current_timestamp

1670857532.9559953

In [ ]:
# assigned regular string date
date_time = datetime.datetime(2021, 8, 24, 00, 00)
 
# print regular python date&time
print("date_time =>",date_time)
 
# displaying unix timestamp after conversion
print("unix_timestamp => ",
      (time.mktime(date_time.timetuple())))

date_time => 2021-08-24 00:00:00
unix_timestamp =>  1629763200.0


In [ ]:
# assigned regular string date
date_time = datetime.datetime(2021, 9, 14, 00, 00)
 
# print regular python date&time
print("date_time =>",date_time)
 
# displaying unix timestamp after conversion
print("unix_timestamp => ",
      (time.mktime(date_time.timetuple())))

date_time => 2021-09-14 00:00:00
unix_timestamp =>  1631577600.0


### B. Get block number by Etherscan API

https://docs.etherscan.io/api-endpoints/blocks#get-block-number-by-timestamp
https://www.geeksforgeeks.org/

Get your API Key at: https://etherscan.io/myapikey

In [ ]:
pip install etherscan-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#@title Your API_Key
API_Key = "API_KEY" #@param {type:"string"}



In [ ]:
from etherscan import Etherscan
eth = Etherscan('API_KEY')

In [ ]:
start = eth.get_block_number_by_timestamp(timestamp="1629763200", closest="before")
start

'13084679'

In [ ]:
end = eth.get_block_number_by_timestamp(timestamp="1631577600", closest="after")
end

'13220489'

In [ ]:
import numpy as np
duration = int(end)-int(start)
duration

135810

C. Query the waiting time (delay) data during the NFT drops period

In [ ]:
# Connect to an existing database
with psycopg.connect(POSTGRES_DB_URL, autocommit=True) as conn:

    # Open a cursor to perform database operations
    with conn.cursor() as cur:

        # Execute a command: this creates a new table
        sql=cur.execute("select delay, included_in_block_num from processed_transactions WHERE included_in_block_num >= 13084679 and included_in_block_num <= 13220489").fetchall()
        dat2 =pd.DataFrame(sql, columns=selected_columns)

In [ ]:
wt_NFT = dat2
wt_NFT

,delay,included_in_block_num
